In [ ]:
from Discrete import *
import numpy as np

import seaborn as sns
plt.style.use("ggplot")
plt.rcParams["font.family"] = "sans-serif"
sns.set_palette("dark")
sns.set_context("talk")

Q = np.diag((10., 10., 1., 1.))
R = np.array([1.])

eq_state = np.array([np.pi, 0., 0., 0.])
eq_input = 0.

C = np.eye(4)
D = np.zeros((4, 1))

matrices, ranks, eigenvalues, freq, delta, Ad_tmotor, Bd_tmotor = controllabilityOverFreq(1, 1000, eq_state, eq_input, "Acrobot/acrobot_tmotors.urdf", Q, R)
matrices2, ranks2, eigenvalues2, freq2, delta2, Ad_mj, Bd_mj = controllabilityOverFreq(1, 1000, eq_state, eq_input, "Acrobot/acrobot_mjbots.urdf", Q, R)

freq = np.flip(freq)
freq2 = np.flip(freq2)

show=True

# print((eigenvalues[:5, :] - eigenvalues2[:5, :]))
# print(eigenvalues[:5, :])
# print(eigenvalues2[:5, :])

fig, (ax1, ax2) = plt.subplots(2, 1)

ax1.plot(1/eigenvalues[:, 0], eigenvalues[:, 1], label=r"$\lambda_{1}$")
ax1.plot(1/eigenvalues[:, 0], eigenvalues[:, 2], label=r"$\lambda_{2}$")
ax1.plot(1/eigenvalues[:, 0], eigenvalues[:, 3], label=r"$\lambda_{3}$")
ax1.plot(1/eigenvalues[:, 0], eigenvalues[:, 4], label=r"$\lambda_{4}$")
#plt.title("Eigenvalues discrete Acrobot TMotors")
#plt.xlabel("frequency [Hz]")
#plt.ylabel(r"$|\lambda|$")
plt.legend()
plt.tight_layout()
if show:
    # plt.show()
    pass
else:
    plt.savefig("Figures/discrete_acrobot_tmotors.png", dpi=400)
    plt.close()


ax2.plot(1/eigenvalues2[:, 0], eigenvalues2[:, 1], label=r"$\lambda_{1}$")
ax2.plot(1/eigenvalues2[:, 0], eigenvalues2[:, 2], label=r"$\lambda_{2}$")
ax2.plot(1/eigenvalues2[:, 0], eigenvalues2[:, 3], label=r"$\lambda_{3}$")
ax2.plot(1/eigenvalues2[:, 0], eigenvalues2[:, 4], label=r"$\lambda_{4}$")
#plt.title("Eigenvalues discrete acrobot MJbots")
#plt.xlabel("frequency [Hz]")
#plt.ylabel(r"$|\lambda|$")
plt.legend()
plt.tight_layout()
if show:
    plt.show()
else:
    plt.savefig("Figures/discrete_acrobot_mjbots.png", dpi=400)
    plt.close()

# Simulation of the discrete system
sim_tmotor = SimWithRate(Ad_tmotor, Bd_tmotor, C, D, Q, R, 500., np.array([np.pi, 0., 0., 0.]), eq_state, eq_input)
t_sim_tmotor = np.linspace(0, 10, np.shape(sim_tmotor)[1])

plt.plot(t_sim_tmotor, sim_tmotor[0, :])
# plt.ylim([-7, 7])
plt.show()

sim_mj = SimWithRate(Ad_mj, Bd_mj, C, D, Q, R, 500., np.array([np.pi, 0., 0., 0.]), eq_state, eq_input)
t_sim_mj = np.linspace(0, 10, np.shape(sim_mj)[1])

plt.plot(t_sim_tmotor, sim_tmotor[0, :])
# plt.ylim([-7, 7])
plt.show()

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.scatter(delta, ranks)
ax2.scatter(delta2, ranks2)
plt.show()

dt = 1/500.
A_cont, B_cont = makePlant(eq_state, eq_input, "Acrobot/acrobot_tmotors.urdf")
A_disc, B_disc = makePlantDiscrete(eq_state, eq_input, "Acrobot/acrobot_tmotors.urdf", dt)
# print(A_cont, B_cont)
A_test = expm(dt*(A_cont))
B_test = np.linalg.inv(A_cont) @ (A_test - np.eye(4)) @ B_cont

# print(A_test - A_disc)
# print(B_test - B_disc)

K_test, S_test = DiscreteTimeLinearQuadraticRegulator(A_disc, B_disc, Q, R)
eigs = np.linalg.eigvals(A_disc-B_disc@K_test)

# print(dt, eigs)
print(A_disc-B_disc@K_test)
# print(dlqr(A_disc, B_disc, Q, R)[2])
# print(eigenvalues[-1, :])
# print(S_test)

steps = np.linspace(0, 10, 5001)
x0 = np.array([np.pi, 0, 0, 0])
x = np.vstack((x0, np.zeros((len(steps)-1, len(eq_state)))))
for step in range(len(steps)-1):
    # x[step+1, :] = x[step, :] + ((A_disc-B_disc@K_test)@(x[step, :] - eq_state).reshape((4, 1))).reshape(4,)
    u = -B_disc@(K_test@(x[step, :] - eq_state).reshape((4, 1)))
    u[u > 10] = 10
    u[u < -10] = -10

    x[step+1, :] = (A_disc@(x[step, :] - eq_state).reshape((4, 1)) + B_disc*u).reshape(4,)
    # print(B_disc@K_test@(x[step, :] - eq_state))
    
plt.plot(steps, x[:, 0])
plt.plot(steps, x[:, 1])
plt.show()
